<a href="https://colab.research.google.com/github/name1929/Kaggle/blob/master/%EB%8D%B0%EC%9D%B4%EC%BD%98_%EC%8A%A4%ED%81%AC%EB%A6%BD%ED%8A%B8%EC%9E%AC%EA%B5%AC%EC%84%B1(19_12_06)_backup.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. 패키지 및 데이터 로드



## 1-1. 패키지 로드

In [0]:
import pandas as pd # 데이터 전처리
import numpy as np # 데이터 전처리
import random #데이터 전처리
from pandas import DataFrame #데이터 전처리
from collections import Counter #데이터 전처리

from tqdm import tqdm #시간 측정용

from sklearn.feature_extraction.text import CountVectorizer # model setting
from sklearn.model_selection import train_test_split  # model setting

from sklearn.naive_bayes import MultinomialNB  # model 관련
from sklearn.metrics import roc_auc_score  # model 성능 확인

In [0]:
from google.colab import drive

drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


## 1-2. 데이터 로드


In [0]:
!ls "/content/gdrive/My Drive/14th data" #현재 drive 경로 안의 파일 or 폴더 표시

14th_baseline_multi.csv    public_test.csv	    train.csv
Mecab-ko-for-Google-Colab  submission_제출양식.csv


In [0]:
cd /content/gdrive/My Drive/14th data

/content/gdrive/My Drive/14th data


In [0]:
train = pd.read_csv("train.csv") #해당 14th data의 csv 파일 중 train.csv 불러오기

train.head(2)

,id,year_month,text,smishing
0,0,2017-01,XXX은행성산XXX팀장입니다.행복한주말되세요,0
1,1,2017-01,오늘도많이웃으시는하루시작하세요XXX은행 진월동VIP라운지 XXX올림,0


In [0]:
test = pd.read_csv("public_test.csv")

test.head(2)

,id,year_month,text
0,340000,2019-01,XXX고객님! 안녕하세요? 새롭게 시작하는 한 주 행복 가득하시길 기원합니다. 지난...
1,340001,2019-01,긴급 안내 XXX은행 가락동 지점 - 헬리오XXX 기본XXX 대출이자를 ...


In [0]:
submission=pd.read_csv("submission_제출양식.csv")

submission.head(2)

,id,smishing
0,340000,1.000000e-09
1,340001,1.000000e-09


In [0]:
train.shape, test.shape, submission.shape

((295945, 4), (1626, 3), (1626, 2))

# 2. 데이터 샘플링 및 분할

## 2-1. Training set to (train_x, val_x, train_y, val_y)


NLP 특성상 해당 train data를 모두 tokenizing 한 뒤, CountVectorizer를 하게 되면,

 colab 상에서는 과하게 ram을 잡아먹어 다운이 되는 경우가 생기기 때문에, 스미싱 비율에 맞게 under sampling을 수행하였다.

In [0]:
Counter(train['smishing'])

Counter({0: 277242, 1: 18703})

In [0]:
Counter(train['smishing'])[0] * 0.1 ,  Counter(train['smishing'])[1] * 0.1

(27724.2, 1870.3000000000002)

In [0]:
random.seed(2019)
all_nsm_list = list(train[train['smishing']==0].index)
all_sm_list = list(train[train['smishing']==1].index)

In [0]:
sample_nsm_list = random.sample(all_nsm_list, 27724) # sampling : 10% 
sample_sm_list = train_sm = random.sample(all_sm_list, 1870) # sampling : 10% 

In [0]:
training_set = train.iloc[sample_nsm_list + sample_sm_list, :].reset_index(drop=True)
training_set.head(2)
#train_y = DataFrame(train.iloc[sample_nsm_list + sample_sm_list, 3].reset_index(drop=True), columns=['smishing'])

,id,year_month,text,smishing
0,97591,2017-05,고객님 업무처리해드린 XXX은행 강남대로 이XXX입니다.다시한번 거래 감사드리고 혹...,0
1,151179,2017-08,안녕하세요? 오늘 하루도 잘 보내셨지요?금요일도 아닌데 문자 드려서 깜놀 하셨나요갑...,0


In [0]:
X = training_set[['id', 'year_month', 'text']]
Y = training_set[['smishing']]

train_test_split 함수

arrays : 분할시킬 데이터를 입력 (Python list, Numpy array, Pandas dataframe 등..)

test_size : 테스트 데이터셋의 비율(float)이나 갯수(int) (default = 0.25)

train_size : 학습 데이터셋의 비율(float)이나 갯수(int) (default = test_size의 나머지)

random_state : 데이터 분할시 셔플이 이루어지는데 이를 위한 시드값 (int나 RandomState로 입력)

shuffle : 셔플여부설정 (default = True)

stratify : 지정한 Data의 비율을 유지한다. 예를 들어, Label Set인 Y가 25%의 0과 75%의 1로 이루어진 Binary Set일 때, stratify=Y로 설정하면 나누어진 데이터셋들도 0과 1을 각각 25%, 75%로 유지한 채 분할된다.

In [0]:
from sklearn.model_selection import train_test_split
train_x, val_x, train_y, val_y = train_test_split(X, Y, test_size = 0.25, random_state=2019, stratify=Y) #stratify=Y 하면 Label 비율도 유지

In [0]:
train_x.head(2)

,id,year_month,text
23279,236789,2018-03,XXX 고객님 어제업무처리해준 XXX호평지점 XXX차장입니다 항상 가족처럼 편안하게...
27687,129437,2017-06,XXX 안산종합금융센터 XXX 차장입니다.XXX은행안산XXX올림


In [0]:
train_y.head(2)

,smishing
23279,0
27687,0


In [0]:
Counter(train_y['smishing']), Counter(val_y['smishing'])

(Counter({0: 20793, 1: 1402}), Counter({0: 6931, 1: 468}))

## 2-2. Test 데이터 전처리(test_x, test_y)

In [0]:
test.head(2)

,id,year_month,text
0,340000,2019-01,XXX고객님! 안녕하세요? 새롭게 시작하는 한 주 행복 가득하시길 기원합니다. 지난...
1,340001,2019-01,긴급 안내 XXX은행 가락동 지점 - 헬리오XXX 기본XXX 대출이자를 ...


In [0]:
test['smishing'] = 2

In [0]:
test.head(2)  

,id,year_month,text,smishing
0,340000,2019-01,XXX고객님! 안녕하세요? 새롭게 시작하는 한 주 행복 가득하시길 기원합니다. 지난...,2
1,340001,2019-01,긴급 안내 XXX은행 가락동 지점 - 헬리오XXX 기본XXX 대출이자를 ...,2


In [0]:
test_x = DataFrame(test['text'])
test_y = DataFrame(test['smishing'])

In [0]:
train_x.shape, train_y.shape, val_x.shape, val_y.shape, test_x.shape, test_y.shape


((22195, 3), (22195, 1), (7399, 3), (7399, 1), (1626, 1), (1626, 1))

# 3. Tokenizing

## 3-1. Mecab 다운로드 

현재 다양한 자연어 처리 패키지 중에서 mecab는 윈도우에서는 설치가 힘든 패키지 중 하나이다.

이를 극복하기 위해, colab에서 mecab를 설치하고 활용할 수 있도록 colab 파일을 공유하고자 한다.

In [0]:
#! git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git

In [0]:
ls

14th_baseline_multi.csv     public_test.csv          train.csv
Mecab-ko-for-Google-Colab/  submission_제출양식.csv


In [0]:
cd Mecab-ko-for-Google-Colab/

/content/gdrive/My Drive/14th data/Mecab-ko-for-Google-Colab


In [0]:
ls

images/  install_mecab-ko_on_colab190912.sh  README.md


In [0]:
%%time
! bash install_mecab-ko_on_colab190912.sh #이 코드 실행하지 않으면, 아래 코드에서 ModuleNotFoundError: No module named 'konlpy'

Installing konlpy.....
     |████████████████████████████████| 19.4MB 121kB/s 
     |████████████████████████████████| 2.7MB 38.0MB/s 
     |████████████████████████████████| 92kB 11.4MB/s 
  Found existing installation: tweepy 3.6.0
    Uninstalling tweepy-3.6.0:
      Successfully uninstalled tweepy-3.6.0
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3
Done
Installing mecab-0.996-ko-0.9.2.tar.gz.....
from https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
--2019-12-06 00:17:23--  https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
Resolving bitbucket.org (bitbucket.org)... 18.205.93.1, 18.205.93.0, 18.205.93.2, ...
Connecting to bitbucket.org (bitbucket.org)|18.205.93.1|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://bbuseruploads.s3.amazonaws.com/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz?

## 3-2. Mecab 사용하여 tokenizing(train_doc, val_doc, test_doc)


In [0]:
import konlpy
from konlpy.tag import Mecab

tokenizer = Mecab() # setting tokenizer using Mecab()

list comprehension 예시


In [0]:
evens = [x * 2 for x in range(11)] #list comprehension 예시
evens

[0, 2, 4, 6, 8, 10, 12, 14, 16, 18, 20]

아래는 list comprehension 사용해서 
리스트 안에 튜플(tokenizer.pos(x), y) 값을 담으려는 것


In [0]:
zip(train_x['text'], train_y['smishing'])

In [0]:
tqdm(zip(train_x['text'], train_y['smishing']))


0it [00:00, ?it/s]

0it [00:00, ?it/s]

"train_x, val_x, test_x" 문장이 들어있는 df => tokenizing 됨

"train_y, val_y, test_y" 0 or 1이 들어있는 df => token들과 튜플에 같이 담김. 

In [0]:
#list comprehension 형태

train_doc = [ ( tokenizer.pos(x), y ) for x, y in tqdm( zip( train_x['text'], train_y['smishing'] ) )  ] # Mecab를 활용하여 text를 토큰화 시킴
val_doc = [ ( tokenizer.pos(x), y ) for x, y in tqdm( zip( val_x['text'], val_y['smishing'] ) )  ]
test_doc = [ ( tokenizer.pos(x), y ) for x, y in tqdm( zip( test_x['text'], test_y['smishing'] ) )  ]



0it [00:00, ?it/s]

242it [00:00, 2379.54it/s]

469it [00:00, 2343.09it/s]

712it [00:00, 2366.33it/s]

975it [00:00, 2438.08it/s]

1206it [00:00, 2395.59it/s]

1438it [00:00, 2362.08it/s]

1716it [00:00, 2469.08it/s]

1956it [00:00, 2447.83it/s]

2206it [00:00, 2460.55it/s]

2480it [00:01, 2537.69it/s]

2729it [00:01, 2443.70it/s]

2971it [00:01, 2284.06it/s]

3212it [00:01, 2305.89it/s]

3469it [00:01, 2376.20it/s]

3719it [00:01, 2412.02it/s]

3963it [00:01, 2416.65it/s]

4252it [00:01, 2533.44it/s]

4508it [00:01, 2472.53it/s]

4757it [00:01, 2475.50it/s]

5006it [00:02, 2476.83it/s]

5255it [00:02, 2478.67it/s]

5519it [00:02, 2523.01it/s]

5772it [00:02, 1605.56it/s]

6002it [00:02, 1761.39it/s]

6213it [00:02, 1851.24it/s]

6446it [00:02, 1972.39it/s]

6664it [00:02, 2008.48it/s]

6901it [00:03, 2104.44it/s]

7139it [00:03, 2178.66it/s]

7399it [00:03, 2289.84it/s]

7636it [00:03, 2289.65it/s]

7871it [00:03, 2297.98it/s]

8113it [00:03, 2333.25it/s]

8372it [00:03, 2402.57it/

# 4. Tokenizing 후 정제 

## 4-1. Token 정제 코드(데이콘 제공)

단어가 중복해서 들어가는 문제가 있어, 새로 만듦.

필요없는 단어를 없애는 함수

In [0]:
#train_doc[1]

In [0]:
#stopwords = ['XXX', '.', '을', '를', '이', '가', '-', '(', ')', ':', '!', '?', ')-', '.-', 'ㅡ', 'XXXXXX', '..', '.(', '은', '는'] #필요없는 단어 리스트

#def get_couple(_words): #필요없는 단어들 없애는 함수 / _words는 lwords[0], lwords[1], ... , 등으로 들어감. 즉 ([('(', 'SSO'), ('광고', 'NNG'), .....], 1) 이렇게 들어감.

#    global stopwords
#    _words = [x for x in _words if x[0] not in stopwords] #x[0]은 lwords[0][0][0], lwords[0][1][0], lwords[0][2][0] 등이다. 즉 '(', '광고', ')', 'S' 등
#    l = len(_words)
#    for i in range(l-1):
#        yield _words[i][0], _words[i+1][0]

필요없는 단어들을 제거하면서 동시에 모형에 사용하기 위한, X_train, X_val, X_test, Y_train, Y_val를 생성한다

tokenizing 한 뒤, 품사?('SSO', 'NNG' 등) 버린다.

In [0]:
#X_train, Y_train = [], []
#for lwords in train_doc:
#    Y_train.append(lwords[1]) #train_doc[lwords][1]
    
#    temp = []
#    for x, y in get_couple(lwords[0]): #train_doc[lwords][0]
#        temp.append("{}.{}".format(x, y))
#    
#    X_train.append(" ".join(temp))

In [0]:
#X_val, Y_val = [], []
#for lwords in val_doc:
#    Y_val.append(lwords[1]) 
    
#    temp = []
#    for x, y in get_couple(lwords[0]):
#        temp.append("{}.{}".format(x, y))
    
#    X_val.append(" ".join(temp))

In [0]:
#X_test = []
#for lwords in test_doc:
    
#    temp = []
#    for x, y in get_couple(lwords[0]):

#        temp.append("{}.{}".format(x, y))
    
#    X_test.append(" ".join(temp))

## 4-2. Token 정제 코드(내가 만듦)


In [0]:
#내가 만든 코드(임시)
#X_train, Y_train = [], []
#for i in train_doc:
#  Y_train.append(i[1])

#  tmp =[]
#  for j in range(len(i[0])):
#    tmp.append(i[0][j][0])
  
#  X_train.append(" ".join(tmp))

In [0]:
#내가 만든 코드 
stopwords = ['XXX', '.', '을', '를', '이', '가', '-', '(', ')', ':', '!', '?', ')-', '.-', 'ㅡ', 'XXXXXX', '..', '.(', '은', '는'] #필요없는 단어 리스트

X_train, Y_train = [], []
for i in train_doc:
  Y_train.append(i[1])

  tmp =[]
  for j in range(len(i[0])):
    if i[0][j][0] not in stopwords:
      tmp.append(i[0][j][0])
  
  X_train.append(" ".join(tmp))

In [219]:
len(X_train), len(Y_train)

(22195, 22195)

In [233]:
X_train[0:5]

['고객 님 어제 업무 처리 해 준 호평 지점 차장 입니다 항상 가족 처럼 편안 하 게 모시 겠 습니다 혹 본사 고객 만족 도 전화 시 매우 동의 전반 적 만족 꼭 부탁 드리 겠 습니다 미세먼지 심한 데 건강 조심 하 시 고 행복 한 하루 되 세요 은행 호평 올림',
 '안산 종합 금융 센터 차장 입니다 행안 산 올림',
 '서산 지점 거래 해 주 셔서 진심 으로 감사 드립니다 행복 한 하루 되 세요',
 '꽃 들 의 미소 아름다웠 던 봄날 이젠 초여름 의 기운 넘치 네요 흐르 땀 식혀 주 시원 한 바람 처럼 기분 좋 일 많 5 월 의 하루 되 시 길 바랍니다 내 몸 에서 떨어지 지 않 게 건강 도 잘 챙기 시 구요 양 올림',
 '리브 통해 환전 하 실 경우 환율 최대 90 % 까지 우대 해 드립니다 USD 등 주요 통화 기준 또한 가끔 씩 친구 들 에게 돈 보내 줘야 할 때 도 일일이 계좌 번호 물어볼 필요 없이 카톡 에 친구 등록 만 되 어 있 으면 일 일 최대 30 만 원 까지 바로 송금 가능 합니다 너무나 편리 한 리브 앱 스토어 또는 플레이 스토어 에서 리브 로 검색 하 시 면 바로 다운로드 가능 합니다 가입 하 실 때 제 직원 번호 잊 지 마시 고 꼭 입력 부탁 드립니다 행여 수 올림']

In [0]:
stopwords = ['XXX', '.', '을', '를', '이', '가', '-', '(', ')', ':', '!', '?', ')-', '.-', 'ㅡ', 'XXXXXX', '..', '.(', '은', '는'] #필요없는 단어 리스트

X_val, Y_val = [], []
for i in val_doc:
  Y_val.append(i[1])

  tmp =[]
  for j in range(len(i[0])):
    if i[0][j][0] not in stopwords:
      tmp.append(i[0][j][0])
  
  X_val.append(" ".join(tmp))

In [221]:
len(X_val), len(Y_val)

(7399, 7399)

In [232]:
X_val[0:5]

['광고 은행 의 모바일 서비스 리브 Liiv 안 내 고객 님 나 만 의 생활 금융 플랫 폼 리브 Liiv 에서 다양 한 혜택 만나 보 세요 리브 의 혜택 수수료 공인 인 증서 보안 카드 없이 상대방 의 이름 만 으로 간편 하 게 리브 머니 보내 기 누구 나 환율 우대 최고 90 % 까지 받 수 있 리브 환전 할인 된 금액 으로 기프 티 콘 선물 하 기 계산 부터 나누 기 까지 간편 하 게 리브 더치페이 그 외 다양 한 기능 있 으니 다운 받 아 새로운 금융 생활 라이프 즐겨 보 세요 앱 설치 바로 기 안드로이드 https goo glxpVFzE 아이폰 https appsto rekrAnpidb i 저 추천 해 주 세요 권유 직원 계장 추천 무료 수신 거부',
 '언제나 잊 지 않 고 찾 아 주 셔서 감사 합니다 행복 한 하루 되 세요 올림',
 '방배 남 입니다 맑 햇살 과 행복 가득 담 하루 선물 합니다',
 '고객 님 3 월 에 꽃 되 고 싶 다 시 윤보영 3 월 에 꽃 되 고 싶 다 마음 에서 고운 향기 나 꽃 나 보 고 다가오 바 랍 에게 미소 로 안 부를 전 하 고 싶 다 안부 에 향기 나누 여유 담겼 으면 좋 겠 다 여유 속 에서 한 번 쯤 꽃 심 마음 도 헤아려 보 아야 겠다 꽃 인 나 모두 알아볼 수 있 게 아름다운 꽃 되 고 싶 다 꽃 보 사람 마다 가슴 에 행복 담기 행운 의 꽃 었 으면 좋 겠 다 꽃 인 내 행복 한 것 처럼 모두 행복 한 꽃 되 었 으면 좋 겠 다 고객 님 무도 행복 할 3 월 우리 함께 맞이 해요 은행 부산 종합 금융 센터 올림',
 '파란 하늘 처럼 기분 좋 하루 되 세요 매우 동의 부탁 드립니다 장유 대리']

In [0]:
stopwords = ['XXX', '.', '을', '를', '이', '가', '-', '(', ')', ':', '!', '?', ')-', '.-', 'ㅡ', 'XXXXXX', '..', '.(', '은', '는'] #필요없는 단어 리스트

X_test = []
for i in test_doc:
  
  tmp =[]
  for j in range(len(i[0])):
    if i[0][j][0] not in stopwords:
      tmp.append(i[0][j][0])
  
  X_test.append(" ".join(tmp))

In [222]:
len(X_test)

1626

In [228]:
X_test[0:5]

['고객 님 안녕 하 세요 새롭 게 시작 하 한 주 행복 가득 하 시 길 기원 합니다 지난주 글로벌 증시 파월 미 연준 의장 통화 정책 에 대해 완화 적 인 태도 변화 보인 것 주초 투자 심리 개선 으로 이어지 며 큰 폭 상승 마감 했 습니다 더불 어 미 중 차관 급 무역 협상 순조 롭 게 진행 된 점 도 긍정 적 었 습니다 원 달러 환율 주간 7 5 원 으로 하락 하 가운데 외국인 투자자 의 국내 증시 순 매수 규모 8 천억 상회 하 며 수급 개선 된 점 도 주목 해 볼 필요 있 습니다 금주 시장 미국 금융 주 실적 발표 시작 으로 본격 적 인 실적 시즌 에 진입 하 게 됩니다 지난주 애플 삼성전자 등 주요 기업 들 의 1 분기 실적 전망 하향 조정 있 었 던 점 참고 한다면 올해 글로벌 성장 둔화 기조 속 에서 기업 들 의 순익 감소 추이 관심 있 게 살펴보 실 필요 있 어 보 입니다 주 후반 에 브렉 시트 영국 하원 표결 예정 되 어 있 습니다 추워진 날씨 에 건강 유의 하 시 고 좋 한 주 되 세요 감사 합니다 은행 송도 센터 올림',
 '긴급 안내 은행 가락동 지점 헬리오 기본 대출 이자 조합 에서 납부 했었으나 2018 년 12 월 25 일부 로 종료 되 었 습니다 따라서 2019 년 1 월 25 일 부터 매월 25 일 중도금 대출 이자 계약자 님 께서 직접 납부 하 셔야 됩니다 방법 안내 1 자동 체 등록 은행 영업 점 본인 방문 신분증 지참 직원 에게 대출 계좌 대출 자 자동 납부 로 계좌 간 자동 체 등록 요청 인터넷 뱅킹 공인 인 증서 로그인 후 자동 체 등록 2 자동 체 미 등록 고객 2019 1 25 일 까지 가까운 은행 영업 점 방문 하 여 대출 자 납부 3 대출 이자 매월 25 일 직접 납부 하 셔야 되 사항 으로 자동 체 등록 간곡히 권장 합니다 4 매월 25 일 대출 자 미납 시 대출 부분 에 대하 여 연체 자 및 신용 상 의 불 이익 발생 될 수 있 사오니 점 꼭 유의 하 시 길 바랍니다 5 현재 기본 대출 보유 하 고 고객 님 

# 5. Token ▶ CountVectorizing + array 변환

1.   CountVecotrizer를 사용하여서 학습 데이터를 바탕으로 벡터화를 시키고 이를 val data, test data에 적용한다

2.   과정: train data Token ▶ CountVectorizing + array 변환 ▶ val data, test data 적용 + array 변환



## 5-1. CountVectorizer 생성

In [0]:
v=CountVectorizer() 

## 5-2. train data token에 적합한 모델 생성

In [242]:
v.fit(X_train)

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=None, min_df=1,
                ngram_range=(1, 1), preprocessor=None, stop_words=None,
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, vocabulary=None)

## 5-3. "train, val, test"에 모델 적용 + array 변환

In [247]:
vec_x_train = v.transform(X_train).toarray()
vec_x_train 

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [248]:
vec_x_val= v.transform(X_val).toarray()
vec_x_val

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [249]:
vec_x_test= v.transform(X_test).toarray()
vec_x_test

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [250]:
vec_x_train.shape, vec_x_val.shape, vec_x_test.shape

((22195, 16583), (7399, 16583), (1626, 16583))

In [251]:
vec_x_train[0]

array([0, 0, 0, ..., 0, 0, 0])

# 6. 분석 모델 적용

## 6-1. 분류분석(naive bayes 사용) 

naive bayes중 MultinomialNB를 사용하여서 학습 모형을 만들고 이를 test data에 적용한다
[Naive bayes 활용 Pipeline 구축](https://datascienceschool.net/view-notebook/c19b48e3c7b048668f2bb0a113bd25f7/)


### 6-1-1. 모델 생성

In [0]:
m1= MultinomialNB()

### 6-1-2. 모델 학습

In [277]:
m1.fit(vec_x_train,Y_train) 

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

### 6-1-3. 예측하기


1.   각 행의 첫 번째 원소는 첫 번째 클래스의 예측 확률이고 두번째 원소는 두 번째 클래스의 예측 확률입니다. 확률이기 때문에 predict_proba의 출력은 항상 0과 1 사이의 값이며 두 클래스에 대한 확률의 합은 항상 1입니다.

2.   [분류분석 참조, 그래디언트 부스팅 모델](https://subinium.github.io/MLwithPython-2-4/)



In [0]:
Y_train_pred = m1.predict_proba(vec_x_train)
Y_train_pred_one = [ i[1]  for i in y_train_pred1] #i[1]: 값이 '1'일 확률

Y_val_pred = m1.predict_proba(vec_x_val)
Y_val_pred_one = [ i[1]  for i in y_val_pred1] #i[1]: 값이 '1'일 확률

Y_test_pred = m1.predict_proba(vec_x_test)
Y_test_pred_one = [ i[1]  for i in y_test_pred1] #i[1]: 값이 '1'일 확률

In [274]:
Y_val[0:5]

[0, 0, 0, 0, 0]

In [273]:
Y_val_pred_one[0:5]

[2.0084385318460284e-87,
 3.751295358117187e-11,
 5.200342664346702e-09,
 2.103913861016981e-53,
 7.907732976590332e-17]

In [0]:
#y_val_pred_trans = []
#for i in y_val_pred1_one:
#  if i>=0.5:
#    i = 1
#    y_val_pred_trans.append(i)
#  else:
#    i = 0
#    y_val_pred_trans.append(i)


In [0]:
#y_val_pred_trans[0:3]

In [0]:
#val_y.head()

In [0]:
#val_y['pred'] = y_val_pred_trans

In [0]:
#val_y.head()

### 6-1-4. 검증셋(Y_val) 스코어 확인


1.   평가 지표(Metric)은 AUC
2.   참가자 하루 최대 제출 횟수 3회



In [0]:
#from sklearn.metrics import classification_report

#y_true = val_y['smishing']
#y_pred = val_y['pred']

#print(classification_report(y_true, y_pred, target_names=['class 0', 'class 1']))

In [280]:
Y_val[0:5]

[0, 0, 0, 0, 0]

In [284]:
Y_val_pred_one[0:5]

[2.0084385318460284e-87,
 3.751295358117187e-11,
 5.200342664346702e-09,
 2.103913861016981e-53,
 7.907732976590332e-17]

In [0]:
import numpy as np

y = np.array(Y_val)
pred = np.array(Y_val_pred_one)

array([0, 0, 0, ..., 0, 0, 0])

In [0]:
from sklearn import metrics
fpr, tpr, thresholds = metrics.roc_curve(y, pred, pos_label=1)

In [297]:
metrics.auc(fpr, tpr)

0.9966529046387651

test data에서 각 text당 스미싱 확률을 구할 수 있으니 이를 제출양식의 smishing 변수에 넣고 csv 파일로 내보낸다

In [307]:
!ls "/content/gdrive/My Drive/14th data" 

'14th_baseline_multi(191206-1).csv'   public_test.csv
 14th_baseline_multi.csv	      submission_제출양식.csv
 Mecab-ko-for-Google-Colab	      train.csv


In [308]:
cd /content/gdrive/My Drive/14th data

/content/gdrive/My Drive/14th data


In [0]:
submission['smishing'] = Y_test_pred_one

In [0]:
submission.to_csv("14th_baseline_multi(191206-1).csv",index=False) #현재 결과물인 output2를 구글 드라이브에 submission_test라는 이름으로 저장

## 6-2. 다른 모델 적용 + 스코어 확인
